In [1]:
# Imports
import sys
import torch
import os
import numpy as np
import time
from torch.distributions.multivariate_normal import MultivariateNormal
from sbi.inference import SMCABC, prepare_for_sbi
from sbi.inference import SNRE_B, prepare_for_sbi


In [2]:

os.chdir('/home/samuel/Documents/projects/seq posterior approx w nf/seq posterior approx w nf dev')

sys.path.append('./')

print(os.getcwd())


/home/samuel/Documents/projects/seq posterior approx w nf/seq posterior approx w nf dev


In [3]:

# Load all utility functions for all methods
import mv_gaussian.low_dim_w_learnable_summary_stats.functions as func

In [4]:
seed = 1
# Set model and generate data

x_o, conj_model, analytical_posterior = func.set_up_model(seed)


/home/samuel/anaconda3/envs/py374/lib/python3.7/site-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [5]:
dim = 2

def simulator(theta):

    N_samples = theta.shape[0]

    x = torch.zeros(N_samples, conj_model.N, dim)

    for i in range(N_samples):
        model_tmp = MultivariateNormal(theta[i], conj_model.model.covariance_matrix)
        x[i, :, :] = model_tmp.rsample(sample_shape=(conj_model.N,))

    # return calc_summary_stats(x), theta #/math.sqrt(5) # div with std of prior to nomarlize data
    return func.flatten(x)


# check simulator and prior
simulator, prior = prepare_for_sbi(simulator, conj_model.prior)


In [ ]:

inference = SMCABC(simulator, prior)

In [6]:

#inference = SNRE_B(simulator, prior, max_num_epochs=50)

# function that builds the network
def build_custom_post_net(batch_theta, batch_x):
    flow_lik, flow_post = func.set_up_networks()

    return flow_post


inference = SNRE_B(simulator, prior)

In [7]:
num_rounds = 4
x_o = x_o.flatten()

posteriors = []
proposal = None

for i in range(num_rounds):
    posterior = inference(num_simulations=2500, proposal=proposal, max_num_epochs=50)
    posteriors.append(posterior)
    proposal = posterior.set_default_x(x_o)

end = time.time()



Neural network successfully converged after 41 epochs.

Neural network successfully converged after 51 epochs.

Neural network successfully converged after 41 epochs.


Generating samples:  46%|████▌     | 1152/2500 [02:41<03:08,  7.15it/s]


KeyboardInterrupt: 

In [ ]:
posteriors


In [ ]:

posterior_sample = posteriors[-1].sample((1000,))




In [7]:
x_o = x_o.flatten()

posterior_2dot5k = inference(x_o,
                          num_particles=200,
                          num_initial_pop = 200,
                          num_simulations=2500,
                          epsilon_decay = 0.8)


In [8]:
posterior_5k = inference(x_o,
                          num_particles=200,
                          num_initial_pop = 200,
                          num_simulations=5000,
                          epsilon_decay = 0.8)

In [53]:
posterior_7dot5k = inference(x_o,
                          num_particles=200,
                          num_initial_pop = 200,
                          num_simulations=7500,
                          epsilon_decay = 0.8)

In [55]:
posterior_10k = inference(x_o,
                          num_particles=200,
                          num_initial_pop = 200,
                          num_simulations=10000,
                          epsilon_decay = 0.8)

In [ ]:
posterior_100k = inference(x_o,
                          num_particles=200,
                          num_initial_pop = 200,
                          num_simulations=100000,
                          epsilon_decay = 0.8)

[tensor(22.9325)]

In [51]:
posterior_sample = posterior_2dot5k.sample((1000,))
print(conj_model.kl_div(analytical_posterior, posterior_sample))

posterior_sample = posterior_5k.sample((1000,))
print(conj_model.kl_div(analytical_posterior, posterior_sample))

posterior_sample = posterior_7dot5k.sample((1000,))
print(conj_model.kl_div(analytical_posterior, posterior_sample))

posterior_sample = posterior_10k.sample((1000,))
print(conj_model.kl_div(analytical_posterior, posterior_sample))


posterior_sample = posterior_100k.sample((1000,))
print(conj_model.kl_div(analytical_posterior, posterior_sample))

179.76021393152843